Importing the Dependencies

In [1]:
# used to make numpy arrays (list in python)
import numpy as np
# used for creating data frames
import pandas as pd
# need to split our original data into training and test data
from sklearn.model_selection import train_test_split
# importing logistic regression model
from sklearn.linear_model import LogisticRegression
# used to check how well the model is performing
from sklearn.metrics import accuracy_score
# used for feature scaling
from sklearn.preprocessing import StandardScaler

Data Collection and Processing

In [2]:
# loading the csv file to a Pandas DataFrame
heart_data = pd.read_csv('/content/HeartDiseaseTrain-Test.csv')

In [3]:

# --- Define Mapping Dictionaries ---

for col in heart_data.select_dtypes(include="object").columns:
    heart_data[col] = heart_data[col].astype(str).str.strip()

# Maps 'Female' and 'Male' text to the required 0 and 1 codes.
sex_map = {
    'Female': 0,  # female : 0
    'Male': 1     # male : 1
}

# Maps the descriptive chest pain text to the codes 1 through 4.
chest_pain_map = {
    'Typical angina': 1,    # Value 1: typical angina
    'Atypical angina': 2,   # Value 2: atypical angina
    'Non-anginal pain': 3,  # Value 3: non-anginal pain
    'Asymptomatic': 4       # Value 4: asymptomatic
}

# Maps fasting blood sugar descriptive text to 0 or 1.
# Assumes 'Lower than 120 mg/ml' is the text used for 0 (False).
fbs_map = {
    'Lower than 120 mg/ml': 0, # (fasting blood sugar <= 120 mg/dl) = 0
    'Greater than 120 mg/ml': 1 # Assuming this text is used for 1 (True)
}

# Maps the ECG result descriptions to codes 0, 1, or 2.
restecg_map = {
    'Normal': 0,
    'ST-T wave abnormality': 1,
    'Left ventricular hypertrophy': 2 # Using the full description for safety
}

# Maps exercise induced angina text to 0 or 1.
exang_map = {
    'No': 0.0, # 0 = no
    'Yes': 1.0 # 1 = yes
}

# Maps the slope descriptions to codes 1, 2, or 3.
slope_map = {
    'Upsloping': 1, # Value 1: upsloping
    'Flat': 2,      # Value 2: flat
    'Downsloping': 3 # Value 3: downsloping
}

# Maps the vessel codes 0, 1, 2, or 3.
vessels_map = {
    'Zero': 0,
    'One': 1,
    'Two': 2,
    'Three': 3,
    'Four': 4
}

# Maps thalassemia blood disorder text to codes 3, 6, or 7.
thal_map = {
    'Normal': 3,
    'Fixed Defect': 6,
    'Reversable Defect': 7 # Make sure the spelling and capitalization match your data exactly!
}

# Check remaining non-numeric columns
print("Object columns after mapping:", heart_data.select_dtypes(include="object").columns.tolist())

# Check if the literal 'No' still exists anywhere
mask_no = (heart_data.astype(str) == "No").any()
print("Columns that still contain 'No':", mask_no[mask_no].index.tolist())

# --- Apply Mappings to the 'heart_data' DataFrame ---

print("Starting text-to-numerical conversion...")

# 1. Sex
heart_data['sex'] = heart_data['sex'].replace(sex_map)

# 2. Chest Pain Type
heart_data['chest_pain_type'] = heart_data['chest_pain_type'].replace(chest_pain_map)

# 3. Fasting Blood Sugar
heart_data['fasting_blood_sugar'] = heart_data['fasting_blood_sugar'].replace(fbs_map)

# 4. Resting ECG
heart_data['rest_ecg'] = heart_data['rest_ecg'].replace(restecg_map)

# 5. Exercise Induced Angina
# Note: Assuming your 'exercise_induced_angina' column still contains 'Yes'/'No' text.
heart_data['exercise_induced_angina'] = heart_data['exercise_induced_angina'].replace(exang_map)

# 6. Slope
heart_data['slope'] = heart_data['slope'].replace(slope_map)

# 7. Vessels Colored By Flourosopy
heart_data['vessels_colored_by_flourosopy'] = heart_data['vessels_colored_by_flourosopy'].replace(vessels_map)

# 8. Thalassemia
heart_data["thalassemia"] = heart_data["thalassemia"].astype(str).str.strip()
heart_data["thalassemia"] = heart_data["thalassemia"].replace("No", np.nan)
heart_data["thalassemia"] = heart_data["thalassemia"].replace(thal_map)
heart_data["thalassemia"] = pd.to_numeric(heart_data["thalassemia"], errors="coerce")

# Convert to numeric (so 3/6/7 become numbers)
heart_data["thalassemia"] = pd.to_numeric(heart_data["thalassemia"], errors="coerce")
print("Conversion complete! Data is now numerical, ready for splitting and encoding.")

# Inspect unique values in 'thalassemia' to diagnose the 'No' string error
print("Unique values in 'thalassemia' after mapping:", heart_data['thalassemia'].unique())

Object columns after mapping: ['sex', 'chest_pain_type', 'fasting_blood_sugar', 'rest_ecg', 'exercise_induced_angina', 'slope', 'vessels_colored_by_flourosopy', 'thalassemia']
Columns that still contain 'No': ['exercise_induced_angina', 'thalassemia']
Starting text-to-numerical conversion...
Conversion complete! Data is now numerical, ready for splitting and encoding.
Unique values in 'thalassemia' after mapping: [ 7.  6.  3. nan]


/tmp/ipython-input-222493321.py:75: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  heart_data['sex'] = heart_data['sex'].replace(sex_map)
/tmp/ipython-input-222493321.py:78: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  heart_data['chest_pain_type'] = heart_data['chest_pain_type'].replace(chest_pain_map)
/tmp/ipython-input-222493321.py:81: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in t

In [4]:
# Checking the error
search_string = 'No'
if search_string in heart_data['thalassemia'].values:
    print(f"'{search_string}' is in thalassemia.")
    print((heart_data["thalassemia"] == "No").sum())


In [5]:
print(heart_data['thalassemia'].iloc[0:30])

0     7.0
1     7.0
2     7.0
3     7.0
4     6.0
5     6.0
6     3.0
7     7.0
8     7.0
9     6.0
10    6.0
11    7.0
12    6.0
13    7.0
14    NaN
15    6.0
16    6.0
17    7.0
18    6.0
19    6.0
20    6.0
21    6.0
22    6.0
23    6.0
24    6.0
25    7.0
26    6.0
27    6.0
28    3.0
29    6.0
Name: thalassemia, dtype: float64


In [6]:
# print the first 5 rows of the dataset
heart_data.head()

,age,sex,chest_pain_type,resting_blood_pressure,cholestoral,fasting_blood_sugar,rest_ecg,Max_heart_rate,exercise_induced_angina,oldpeak,slope,vessels_colored_by_flourosopy,thalassemia,target
0,52,1,1,125,212,0,1,168,0.0,1.0,3,2,7.0,0
1,53,1,1,140,203,1,0,155,1.0,3.1,1,0,7.0,0
2,70,1,1,145,174,0,1,125,1.0,2.6,1,0,7.0,0
3,61,1,1,148,203,0,1,161,0.0,0.0,3,1,7.0,0
4,62,0,1,138,294,1,1,106,0.0,1.9,2,3,6.0,0


In [7]:
# print the last 5 rows of the dataset
heart_data.head()

,age,sex,chest_pain_type,resting_blood_pressure,cholestoral,fasting_blood_sugar,rest_ecg,Max_heart_rate,exercise_induced_angina,oldpeak,slope,vessels_colored_by_flourosopy,thalassemia,target
0,52,1,1,125,212,0,1,168,0.0,1.0,3,2,7.0,0
1,53,1,1,140,203,1,0,155,1.0,3.1,1,0,7.0,0
2,70,1,1,145,174,0,1,125,1.0,2.6,1,0,7.0,0
3,61,1,1,148,203,0,1,161,0.0,0.0,3,1,7.0,0
4,62,0,1,138,294,1,1,106,0.0,1.9,2,3,6.0,0


In [8]:
heart_data.iloc[298:303]

,age,sex,chest_pain_type,resting_blood_pressure,cholestoral,fasting_blood_sugar,rest_ecg,Max_heart_rate,exercise_induced_angina,oldpeak,slope,vessels_colored_by_flourosopy,thalassemia,target
298,35,1,2,122,192,0,1,174,0.0,0.0,3,0,6.0,1
299,52,1,2,120,325,0,1,172,0.0,0.2,3,0,6.0,1
300,46,0,2,105,204,0,1,172,0.0,0.0,3,0,6.0,1
301,51,1,3,94,227,0,1,154,1.0,0.0,3,1,7.0,1
302,55,0,2,132,342,0,1,166,0.0,1.2,3,0,6.0,1


In [9]:
# number of rows and columns in the dataset
heart_data.shape

(1025, 14)

In [10]:
# getting some info about the data
heart_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1025 entries, 0 to 1024
Data columns (total 14 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   age                            1025 non-null   int64  
 1   sex                            1025 non-null   int64  
 2   chest_pain_type                1025 non-null   int64  
 3   resting_blood_pressure         1025 non-null   int64  
 4   cholestoral                    1025 non-null   int64  
 5   fasting_blood_sugar            1025 non-null   int64  
 6   rest_ecg                       1025 non-null   int64  
 7   Max_heart_rate                 1025 non-null   int64  
 8   exercise_induced_angina        1025 non-null   float64
 9   oldpeak                        1025 non-null   float64
 10  slope                          1025 non-null   int64  
 11  vessels_colored_by_flourosopy  1025 non-null   int64  
 12  thalassemia                    1018 non-null   f

In [11]:
# check for missing values
heart_data.isnull().sum()
heart_data = heart_data.dropna()
heart_data.isnull().sum()


,0
age,0
sex,0
chest_pain_type,0
resting_blood_pressure,0
cholestoral,0
fasting_blood_sugar,0
rest_ecg,0
Max_heart_rate,0
exercise_induced_angina,0
oldpeak,0


In [12]:
# statistical measures about the data
heart_data.describe()

,age,sex,chest_pain_type,resting_blood_pressure,cholestoral,fasting_blood_sugar,rest_ecg,Max_heart_rate,exercise_induced_angina,oldpeak,slope,vessels_colored_by_flourosopy,thalassemia,target
count,1018.000000,1018.000000,1018.000000,1018.000000,1018.000000,1018.000000,1018.000000,1018.000000,1018.000000,1018.000000,1018.000000,1018.000000,1018.000000,1018.000000
mean,54.447937,0.696464,1.943026,131.636542,246.253438,0.146365,0.529470,149.187623,0.334971,1.074951,2.385069,0.759332,6.214145,0.513752
std,9.101835,0.460011,1.029886,17.574327,51.676406,0.353646,0.528086,23.006116,0.472212,1.177640,0.618506,1.032432,0.961049,0.500057
min,29.000000,0.000000,1.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,1.000000,0.000000,3.000000,0.000000
25%,48.000000,0.000000,1.000000,120.000000,211.000000,0.000000,0.000000,132.000000,0.000000,0.000000,2.000000,0.000000,6.000000,0.000000
50%,56.000000,1.000000,2.000000,130.000000,240.000000,0.000000,1.000000,152.000000,0.000000,0.800000,2.000000,0.000000,6.000000,1.000000
75%,61.000000,1.000000,3.000000,140.000000,275.750000,0.000000,1.000000,166.000000,1.000000,1.800000,3.000000,1.000000,7.000000,1.000000
max,77.000000,1.000000,4.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,3.000000,4.000000,7.000000,1.000000


In [13]:
# checking the distribution of Target Variable
heart_data['target'].value_counts()

,count
target,
1,523
0,495


1 --> Defective Heart

0 --> Healthy Heart

Splitting the Features and Target

In [14]:
X = heart_data.drop(columns='target', axis=1)
Y = heart_data['target']

In [15]:
print(X)

      age  sex  chest_pain_type  resting_blood_pressure  cholestoral  \
0      52    1                1                     125          212   
1      53    1                1                     140          203   
2      70    1                1                     145          174   
3      61    1                1                     148          203   
4      62    0                1                     138          294   
...   ...  ...              ...                     ...          ...   
1020   59    1                2                     140          221   
1021   60    1                1                     125          258   
1022   47    1                1                     110          275   
1023   50    0                1                     110          254   
1024   54    1                1                     120          188   

      fasting_blood_sugar  rest_ecg  Max_heart_rate  exercise_induced_angina  \
0                       0         1             168    

In [16]:
print(Y)

0       0
1       0
2       0
3       0
4       0
       ..
1020    1
1021    0
1022    0
1023    1
1024    0
Name: target, Length: 1018, dtype: int64


Splitting the data into Training Data and Test Data

In [17]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

In [18]:
print(X.shape, X_train.shape, X_test.shape)

(1018, 13) (814, 13) (204, 13)


Model Training

Logistic Regression

In [19]:
model = LogisticRegression(max_iter=1000)

In [20]:
model.fit(X_train, Y_train)

LogisticRegression(max_iter=1000)

Model Evaluation

Accuracy Score

In [21]:
# accuracy on Training Data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train, X_train_prediction)

In [22]:
print("Accuracy on Training Data:", training_data_accuracy)

Accuracy on Training Data: 0.8648648648648649


In [23]:
# accuracy on Test Data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [24]:
print("Accuracy on Test Data:", test_data_accuracy)

Accuracy on Test Data: 0.8235294117647058


Building a Predictive System

In [25]:
# Checking for target=1 (The person does have heart disease)
input_data1 = (41,0,1,130,204,0,0,172,0,1.4,2,0,2)
# Checking for target=0 (The person does not have heart disease)
input_data2 = (62,0,0,140,268,0,0,160,0,3.6,0,2,2)

# Change the input data to a numpy array
input_data1_as_numpy_array = np.asarray(input_data1)
input_data2_as_numpy_array = np.asarray(input_data2)

# Reshape the numpy array as we are predicting for only one instance
input_data1_reshaped = input_data1_as_numpy_array.reshape(1,-1) # TAKE NOTES ON THIS
input_data2_reshaped = input_data2_as_numpy_array.reshape(1,-1)

prediction1 = model.predict(input_data1_reshaped)
print(prediction1)

if (prediction1[0] == 0):
    print('The Person does not have a Heart Disease')
else:
  print('The Person has Heart Disease')

prediction2 = model.predict(input_data2_reshaped)
print(prediction2)

if (prediction2[0] == 0):
    print('The Person does not have a Heart Disease')
else:
  print('The Person has Heart Disease')

[1]
The Person has Heart Disease
[0]
The Person does not have a Heart Disease


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [26]:
# Other testing
input_data3 = (41,0,1,130,204,0,0,172,0,1.4,2,0,2)

input_data3_as_numpy_array = np.asarray(input_data3)

input_data3_reshaped = input_data3_as_numpy_array.reshape(1,-1)

prediction3 = model.predict(input_data3_reshaped)
print(prediction3)

if (prediction3[0] == 0):
    print('The Person does not have a Heart Disease')
else:
  print('The Person has Heart Disease')

[1]
The Person has Heart Disease


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
